In [1]:
# \bar C̄
# \dot Ċ
# \ddot C̈
using Zygote
using Flux: Optimise

In [2]:
struct MNIST
    train_datas::AbstractArray
    train_labels::AbstractArray
    test_datas::AbstractArray
    test_labels::AbstractArray
end

function load_mnist(filepath="/Users/yanghanlin/Desktop/julia-2-week/dataset/")
    train_label_file = "train-labels-idx1-ubyte"
    train_data_file = "train-images-idx3-ubyte"
    
    test_label_file = "t10k-labels-idx1-ubyte"
    test_data_file = "t10k-images-idx3-ubyte"
    
    train_labels = Array{UInt8, 1}()
    train_datas = []
    
    test_labels = Array{UInt8, 1}()
    test_datas = []
    
    devnull = Array{UInt8, 1}()
    open(filepath*train_label_file, "r") do f
        readbytes!(f, devnull, 8)
        readbytes!(f, train_labels, 60000)
    end
    open(filepath*train_data_file, "r") do f
        readbytes!(f, devnull, 16)
        for x in 1:60000
            temp = Array{UInt8, 1}()
            readbytes!(f, temp, 28*28)
            push!(train_datas, temp)
        end
    end
    open(filepath*test_label_file, "r") do f
        readbytes!(f, devnull, 8)
        readbytes!(f, test_labels, 10000)
    end
    open(filepath*test_data_file, "r") do f
        readbytes!(f, devnull, 16)
        for x in 1:10000
            temp = Array{UInt8, 1}()
            readbytes!(f, temp, 28*28)
            push!(test_datas, temp)
        end
    end
    return MNIST(train_datas, train_labels, test_datas, test_labels)
end

mnist = load_mnist()
println(size(mnist.test_datas), size(mnist.train_datas))

(10000,)(60000,)


In [3]:
sigmoid(𝒛) = 1.0/(1.0+exp(-𝒛))
softmax(data) = [exp(x)/sum(exp.(data)) for x in data]

struct Network
    weights::AbstractArray
    biases::AbstractArray
    activation::Any
end

function initnetwork(sizes::Union{Tuple, AbstractArray}, activation::Any)
    weights = []
    biases = []
    for (x, y) in [(sizes[x], sizes[x+1]) for x in 1:(length(sizes)-1)]
        push!(weights, rand(y, x))
        push!(biases, rand(y))
    end
    return Network(weights, biases, activation)
end

function loss(net::Network, data::AbstractArray, label)
    for x in 1:length(net.weights)
        data = net.activation.(net.weights[x] * data + net.biases[x])
    end
    data = softmax(data)
    # notice : this for-loop must start with 0, because the MNIST'data is 0-9
    onehot = [x == label ? 1.0 : 0.0 for x in 0:(length(net.biases[end])-1)]
    return sum((data - onehot).^2 / length(data))
end

function batch_loss(net::Network, one_batch_datas::AbstractArray, one_batch_labels::AbstractArray)
    total_loss = 0.0
    for x in 1:length(one_batch_datas)
        total_loss += loss(net, one_batch_datas[x], one_batch_labels[x])
    end
    return total_loss / length(one_batch_datas)
end

function shuffle(len, batch_size)
    len % batch_size != 0 && error("batch_size parameter wrong")
    batches = [x for x in 1:len]
    for x in 1:10000
        temp = abs.(rand(Int64, 2)).% len
        temp = temp.+1
        batches[temp[1]], batches[temp[2]] = batches[temp[2]], batches[temp[1]]
    end
    
    return batches
end

#=
# This function will calculate gradient with statical method and then update the network's parameters
# This function has been deprecated by yhl because the effiency of this method is very low
function update(net::Network, one_batch_datas::AbstractArray, one_batch_labels::AbstractArray)
    learning_rate = 0.1
    loss₁ = batch_loss(net, one_batch_datas, one_batch_labels)
    
    # calculate gradient for every parameter
    weights_dx = deepcopy(net.weights)
    biases_dx = deepcopy(net.biases)
    
    for cind in CartesianIndices(net.weights)
        for i in CartesianIndices(net.weights[cind])
            net.weights[cind][i] = net.weights[cind][i] + 0.00001
            loss₂ = batch_loss(net, one_batch_datas, one_batch_labels)
            η = (loss₂ - loss₁) / 0.00001
            weights_dx[cind][i] = -learning_rate * η
            net.weights[cind][i] = net.weights[cind][i] - 0.00001
        end
    end
    
    for bind in CartesianIndices(net.biases)
        for i in CartesianIndices(net.biases[bind])
            net.biases[bind][i] = net.biases[bind][i] + 0.00001
            loss₂ = batch_loss(net, one_batch_datas, one_batch_labels)
            η = (loss₂ - loss₁) / 0.00001
            biases_dx[bind][i] = -learning_rate * η
            net.biases[bind][i] = net.biases[bind][i] - 0.00001
        end
    end
    
    for x in length(net.weights)
        net.weights[x] = net.weights[x].-weights_dx[x]
    end
    for x in length(net.biases)
        net.biases[x] = net.biases[x].-biases_dx[x]
    end
end
=#

function get_gradient(net::Network, one_batch_datas::AbstractArray, one_batch_labels::AbstractArray)
    loss₁ = batch_loss(net, one_batch_datas, one_batch_labels)
    
    # calculate gradient for every parameter
    weights_dx = deepcopy(net.weights)
    biases_dx = deepcopy(net.biases)
    
    zygote_grad = gradient(batch_loss, net, one_batch_dats, one_batch_labels)[1]
    weights_dx = zygote_grad[:weights]
    biases_dx = zygote_grad[:biases]
    
    return weights_dx, biases_dx
end

function train(net::Network, datas::AbstractArray, labels::AbstractArray, mini_batches::Int64)
    
    len = length(datas)
    for sround_index in 1:20
        index_batches = shuffle(len, mini_batches)
        rounds = len / mini_batches
        for x in 0:(rounds-1)
            sind = Int64((x*mini_batches+1))
            eind = Int64((x*mini_batches+mini_batches))
            one_batch_datas = [datas[index_batches[t]] for t in sind:eind]
            one_batch_labels = [datas[index_batches[t]] for t in sind:eind]
            # deprecated!
            #update(net, one_batch_datas, one_batch_labels)
            weights_dx, biases_dx = get_gradient(net, one_batch_datas, one_batch_labels)
            
            
            println("rounds $(sround_index) $(sind)-$(eind) has updated over.")
        end
    end
end

network = initnetwork([784, 300, 100, 10], sigmoid)
#loss(network, mnist.train_datas[3], mnist.train_labels[3])
#train(network, mnist.train_datas, mnist.train_labels, 30)

Network(Any[[0.7237922963923853 0.6554228062811784 … 0.7689862749578893 0.9841248628473738; 0.12985747553274796 0.9497392789374135 … 0.6441773691199373 0.5317987525792796; … ; 0.9344136221247197 0.9514445809523902 … 0.8427660604334992 0.823790077252432; 0.5060860165035468 0.966243961404085 … 0.3899097690675617 0.29574772408865435], [0.5426633288489173 0.025882430873772178 … 0.1876816543149895 0.3954284709640379; 0.4573961128859232 0.45901984126079465 … 0.8199060053623943 0.712877385180714; … ; 0.18754308551472554 0.2651147120442674 … 0.5473242036260049 0.7693437753834615; 0.7064610687340045 0.42908694076219667 … 0.2998091171021007 0.9055932671170113], [0.907470263399416 0.8838042537929625 … 0.9584943699290756 0.4506238723339284; 0.22372925770999363 0.7616421052017557 … 0.7947907201148801 0.6546258637141764; … ; 0.3437928519480855 0.13585848033821257 … 0.6836439216173058 0.3824079704934342; 0.19086759961778998 0.8346970632853368 … 0.3049678678487222 0.8985239654299826]], Any[[0.03596672

In [4]:
# NOTES：这次用Zygote与我自己写的gradient进行比较，看最大差值
# 重要的源代码全部摘录自上面我写的网络
# test for gradient
net_test = initnetwork([20, 10, 5, 2], sigmoid)
data = rand(20)
label = 1
@time zygote_grad = gradient(loss, net_test, data, label)[1]

function my_gradient(loss_function, parameters...)
    net = parameters[1]
    data = parameters[2]
    label = parameters[3]
    loss₁ = loss_function(net, data, label)
    
    # calculate gradient for every parameter
    weights_dx = deepcopy(net.weights)
    biases_dx = deepcopy(net.biases)
    
    for cind in CartesianIndices(net.weights)
        for i in CartesianIndices(net.weights[cind])
            net.weights[cind][i] = net.weights[cind][i] + 0.00001
            loss₂ = loss_function(net, data, label)
            weights_dx[cind][i] = (loss₂ - loss₁) / 0.00001
            net.weights[cind][i] = net.weights[cind][i] - 0.00001
        end
    end
    
    for bind in CartesianIndices(net.biases)
        for i in CartesianIndices(net.biases[bind])
            net.biases[bind][i] = net.biases[bind][i] + 0.00001
            loss₂ = loss_function(net, data, label)
            biases_dx[bind][i] = (loss₂ - loss₁) / 0.00001
            net.biases[bind][i] = net.biases[bind][i] - 0.00001
        end
    end
    return (weights_dx, biases_dx)
end

@time my_grad = my_gradient(loss, net_test, data, label)

#println("zygote gradients:")
#println(zygote_grad[:weights])
#println("my gradients:")
#println(my_grad[1])
diff = zygote_grad[:weights].-my_grad[1]
diff[1] = reshape(diff[1], 200, 1)
diff[2] = reshape(diff[2], 50, 1)
diff[3] = reshape(diff[3], 10, 1)
diff = [diff[1];diff[2];diff[3]].^2
println(max(diff...))


  6.722108 seconds (19.97 M allocations: 1.003 GiB)
  0.321412 seconds (1.02 M allocations: 52.738 MiB)
3.9553869014512385e-15


In [5]:
# NOTES：这次用Zygote与我自己写的batch_gradient进行比较，看最大差值
# 重要的源代码全部摘录自上面我写的网络
# test for batch gradient
net_test2 = initnetwork([20, 10, 5, 2], sigmoid)
one_batch_datas = [rand(20) for x in 100]
one_batch_labels = [abs(rand(Int64))%2 for x in 100]
@time zygote_grad = gradient(batch_loss, net_test2, one_batch_datas, one_batch_labels)[1]

function my_batch_gradient(net::Network, one_batch_datas::AbstractArray, one_batch_labels::AbstractArray)
    loss₁ = batch_loss(net, one_batch_datas, one_batch_labels)
    
    # calculate gradient for every parameter
    weights_dx = deepcopy(net.weights)
    biases_dx = deepcopy(net.biases)
    
    for cind in CartesianIndices(net.weights)
        for i in CartesianIndices(net.weights[cind])
            net.weights[cind][i] = net.weights[cind][i] + 0.00001
            loss₂ = batch_loss(net, one_batch_datas, one_batch_labels)
            weights_dx[cind][i] = (loss₂ - loss₁) / 0.00001
            net.weights[cind][i] = net.weights[cind][i] - 0.00001
        end
    end
    
    for bind in CartesianIndices(net.biases)
        for i in CartesianIndices(net.biases[bind])
            net.biases[bind][i] = net.biases[bind][i] + 0.00001
            loss₂ = batch_loss(net, one_batch_datas, one_batch_labels)
            biases_dx[bind][i] = (loss₂ - loss₁) / 0.00001
            net.biases[bind][i] = net.biases[bind][i] - 0.00001
        end
    end
    
    return (weights_dx, biases_dx)
end

@time my_batch_grad = my_batch_gradient(net_test, one_batch_datas, one_batch_labels)

#println("zygote gradients:")
#println(zygote_grad[:weights])
#println("my gradients:")
#println(my_grad[1])
diff = zygote_grad[:weights].-my_grad[1]
diff[1] = reshape(diff[1], 200, 1)
diff[2] = reshape(diff[2], 50, 1)
diff[3] = reshape(diff[3], 10, 1)
diff = [diff[1];diff[2];diff[3]].^2
println(diff)


  0.434560 seconds (898.25 k allocations: 44.707 MiB)
  0.025957 seconds (48.37 k allocations: 2.476 MiB)
[9.409162501187021e-14; 3.747953336995626e-16; 2.3806047904785495e-13; 3.7342498520762885e-15; 7.61357723984382e-15; 1.7039840103914804e-16; 2.0280137833107334e-15; 1.2627206898808941e-13; 5.082964651534606e-14; 5.6254938208633995e-15; 3.7912161245325983e-13; 5.1397946423112165e-14; 4.485257194913899e-13; 1.1102333569614331e-13; 8.2345435182776e-14; 1.3986377428013403e-12; 2.663607081520241e-13; 1.401569445074848e-13; 7.520117762991672e-14; 1.1673291015354743e-14; 1.5298395969613618e-12; 4.8187709742441606e-14; 3.0074062483607323e-12; 1.6397841699246046e-13; 1.9010434220675373e-13; 8.528891757822658e-13; 2.5218367020621e-13; 1.386842646099919e-12; 6.003082767340809e-13; 7.311773235812413e-14; 1.0118109904807181e-12; 6.794335272311561e-14; 1.6254683719353314e-12; 1.7755444127306509e-13; 1.627964557016037e-13; 1.5792250944957715e-12; 3.534558081943825e-13; 6.553418367101549e-13; 3.04